In [3]:
import sys
import torch
from nnsight import LanguageModel

sys.path.append("../../autointerp/mats_sae_training")

from sae_training.sparse_autoencoder import SparseAutoencoder

torch.set_grad_enabled(False)

In [4]:
from huggingface_hub import hf_hub_download

REPO_ID = "jbloom/GPT2-Small-SAEs"

sae_list = []
n_layers = 12


for layer in range(n_layers):
    filename =  f"final_sparse_autoencoder_gpt2-small_blocks.{layer}.hook_resid_pre_24576.pt"

    save_path = f"../../autointerp/jbloom_dictionaries/{filename}"
    sae = SparseAutoencoder.load_from_pretrained(save_path)
    sae.to("cuda:0")
    
    sae_list.append(sae)

print("Loaded dictionaries")

Loaded dictionaries


In [5]:
model = LanguageModel("openai-community/gpt2", device_map="cuda:0", dispatch=True)

In [7]:
sae_list[8]

SparseAutoencoder(
  (hook_sae_in): HookPoint()
  (hook_hidden_pre): HookPoint()
  (hook_hidden_post): HookPoint()
  (hook_sae_out): HookPoint()
)

In [ ]:
prompt = " to firearms. LEGISLATIVE COUNSEL'S DIGEST AB"

with model.trace(prompt):
    activations = model.transformer.h[8].input[0][0]
    middle = sae_list[8](activations)

    acts = middle[1][:,:,7504]
    acts.save()

In [ ]:
with model.trace(" "):
    activations = model.transformer.h[8].input[0][0]
    middle = sae_list[8](activations)
    middle 

    acts = middle[1][:,:,7504]
    acts.save()